In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Log_Anamoly_Detection/dataset/balanced_log_dataset.csv")
# Convert label
df["Label"] = df["Label"].map({"Success": 0, "Fail": 1})

In [ ]:
# Convert string "[E5,E22,...]" → list
df["Features"] = df["Features"].apply(
    lambda x: x.strip("[]").split(",")
)

vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
X_events = vectorizer.fit_transform(df["Features"])


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
df["TimeInterval"] = df["TimeInterval"].apply(
    lambda x: list(map(float, x.strip("[]").split(",")))
)

df["time_mean"] = df["TimeInterval"].apply(np.mean)
df["time_max"] = df["TimeInterval"].apply(np.max)
df["time_sum"] = df["TimeInterval"].apply(np.sum)
df["time_std"] = df["TimeInterval"].apply(np.std)
df["time_nonzero"] = df["TimeInterval"].apply(lambda x: np.count_nonzero(x))


In [ ]:
X_numeric = df[[
    "Type",
    "Latency",
    "time_mean",
    "time_max",
    "time_sum",
    "time_std",
    "time_nonzero"
]]
X_numeric.isna().sum()


,0
Type,16838
Latency,0
time_mean,0
time_max,0
time_sum,0
time_std,0
time_nonzero,0


In [ ]:
X_numeric.isna().sum()
X_numeric = X_numeric.fillna(0)

from scipy.sparse import hstack
X = hstack([X_events, X_numeric.values])

y = df["Label"]

In [ ]:
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
iso = IsolationForest(
    n_estimators=300,
    contamination=0.35,
    random_state=42,
    n_jobs=-1
)

iso.fit(X_train)




IsolationForest(contamination=0.35, n_estimators=300, n_jobs=-1,
                random_state=42)

In [ ]:
# Isolation Forest output:
#  1  = normal
# -1  = anomaly

y_train_raw = iso.predict(X_train)
y_test_raw = iso.predict(X_test)

# Map: anomaly (-1) → Fail (1), normal (1) → Success (0)
y_train_pred = np.where(y_train_raw == -1, 1, 0)
y_test_pred = np.where(y_test_raw == -1, 1, 0)
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_acc)
print("Testing Accuracy:", test_acc)



Training Accuracy: 0.7804380103934669
Testing Accuracy: 0.7871140142517815


In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification Report:")
print(
    classification_report(
        y_test,
        y_test_pred,
        target_names=["Success", "Fail"]
    )
)





Confusion Matrix:
[[3151  217]
 [1217 2151]]

Classification Report:
              precision    recall  f1-score   support

     Success       0.72      0.94      0.81      3368
        Fail       0.91      0.64      0.75      3368

    accuracy                           0.79      6736
   macro avg       0.81      0.79      0.78      6736
weighted avg       0.81      0.79      0.78      6736

